In [1]:
# Import required libraries
import pandas as pd
import plotly.express as px
import dash
from dash import html
from dash import dcc
#importante agregar estas 
from dash.dependencies import Input, Output, State 
from jupyter_dash import JupyterDash 
import plotly.graph_objects as go 
import plotly.express as px 
from dash import no_update

In [2]:
# Add Dataframe
path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DA0101EN-SkillsNetwork/labs/Data%20files/automobileEDA.csv'
auto_data = pd.read_csv(path,encoding = "ISO-8859-1")
auto_data.head()

symboling  normalized-losses         make aspiration num-of-doors  \
0          3                122  alfa-romero        std          two   
1          3                122  alfa-romero        std          two   
2          1                122  alfa-romero        std          two   
3          2                164         audi        std         four   
4          2                164         audi        std         four   

    body-style drive-wheels engine-location  wheel-base    length  ...  \
0  convertible          rwd           front        88.6  0.811148  ...   
1  convertible          rwd           front        88.6  0.811148  ...   
2    hatchback          rwd           front        94.5  0.822681  ...   
3        sedan          fwd           front        99.8  0.848630  ...   
4        sedan          4wd           front        99.4  0.848630  ...   

   compression-ratio  horsepower  peak-rpm city-mpg highway-mpg    price  \
0                9.0       111.0    5000.0       21          27  13495.0   
1                9.0       111.0    5000.0       21          27  16500.0   
2                9.0       154.0    5000.0       19          26  16500.0   
3               10.0       102.0    5500.0       24          30  13950.0   
4                8.0       115.0    5500.0       18          22  17450.0   

  city-L/100km  horsepower-binned  diesel  gas  
0    11.190476             Medium       0    1  
1    11.190476             Medium       0    1  
2    12.368421             Medium       0    1  
3     9.791667             Medium       0    1  
4    13.055556             Medium       0    1  

[5 rows x 29 columns]

In [3]:
# REVIEW1: Clear the layout and do not display exception till callback gets executed
#app.config.suppress_callback_exceptions = True

In [4]:
#creating a list , pero no se si es util , ya que al final lo agregue manualmente 
#debe existir una manera mas facil
dwv=auto_data["drive-wheels"].unique().tolist()
def listToDict(lstA, lstB):
    zippedLst = zip(lstA, lstB)
    op = dict(zippedLst)
    return op


dw = ['rear-wheel drive', 'front-wheel drive', '4wheel drive']
dwl=listToDict(dwv, dw)

In [5]:
app = JupyterDash(__name__) 

# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True


#Layout Section of Dash

app.layout = html.Div(children=[html.H1('Car Automobile Components', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 24}),


     #outer division starts
     html.Div([
                   # First inner divsion for  adding dropdown helper text for Selected Drive wheels
                    html.Div(
                            html.H2('Drive Wheels Type:', style={'margin-right': '2em'}),
                     ),
                    #Second Inner division for adding 2 inner divisions for 2 output graphs 

                      dcc.Dropdown(
                            id='demo-dropdown',
                        options=[
                             {'label': 'Rear Wheel Drive', 'value': 'rwd'},
                            {'label': 'Front Wheel Drive', 'value': 'fwd'},
                             {'label': 'Four Wheel Drive', 'value': '4wd'}
        ],
        value='rwd'
        ),
                    #Second Inner division for adding 2 inner divisions for 2 output graphs 

                    html.Div([
                
                        html.Div([ ], id='plot1'),
                        html.Div([ ], id='plot2')

                        
                    ], style={'display': 'flex'}),


    ])
    #outer division ends

])
#layout ends

#Place to add @app.callback Decorator
@app.callback([Output(component_id='plot1', component_property='children'),
               Output(component_id='plot2', component_property='children')],
               Input(component_id='demo-dropdown', component_property='value'))


   
#Place to define the callback function .
def display_selected_drive_charts(value):
   

   
   filtered_df = auto_data[auto_data['drive-wheels']==value].groupby(['drive-wheels','body-style'],as_index=False). \
            mean()
        
   filtered_df = filtered_df
   
   fig1 = px.pie(filtered_df, values='price', names='body-style', title="Pie Chart")
   fig2 = px.bar(filtered_df, x='body-style', y='price', title='Bar Chart')
    
   return [dcc.Graph(figure=fig1),
            dcc.Graph(figure=fig2) ]

In [6]:
if __name__ == '__main__':
    app.run_server(mode='jupyterlab', port = 8050, dev_tools_ui=True, debug=True,dev_tools_hot_reload =True, threaded=True)